In [10]:


#Loading liabraries and dataset
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
ticker = yf.Ticker("INFY")
df = ticker.history(start="2022-01-01", end="2024-12-31", interval="1d")

# ---- Technical Indicators ---- #

# Bollinger Bands
def add_bollinger_bands(df, window=20, num_std=2):
    df['MA20'] = df['Close'].rolling(window).mean()
    df['Upper_Band'] = df['MA20'] + num_std * df['Close'].rolling(window).std()
    df['Lower_Band'] = df['MA20'] - num_std * df['Close'].rolling(window).std()
    return df

# MACD
def add_macd(df):
    df['EMA12'] = df['Close'].ewm(span=12, adjust=False).mean()
    df['EMA26'] = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = df['EMA12'] - df['EMA26']
    df['signalline'] = df['MACD'].ewm(span=9, adjust=False).mean()
    return df

# RSI
def add_rsi(df, window=14):
    delta = df['Close'].diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(window).mean()
    avg_loss = loss.rolling(window).mean()
    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))
    return df

# Stochastic Oscillator
def add_stochastic(df, k_period=14, d_period=3):
    low_min = df['Low'].rolling(k_period).min()
    high_max = df['High'].rolling(k_period).max()
    df['%K'] = 100 * ((df['Close'] - low_min) / (high_max - low_min))
    df['%D'] = df['%K'].rolling(d_period).mean()
    return df

# ATR
def add_atr(df, period=14):
    df['H-L'] = df['High'] - df['Low']
    df['H-PC'] = abs(df['High'] - df['Close'].shift(1))
    df['L-PC'] = abs(df['Low'] - df['Close'].shift(1))
    df['TR'] = df[['H-L', 'H-PC', 'L-PC']].max(axis=1)
    df['ATR'] = df['TR'].rolling(period).mean()
    return df

# VWAP
def add_vwap(df):
    df['TP'] = (df['High'] + df['Low'] + df['Close']) / 3
    df['Cumulative_TP_Volume'] = (df['TP'] * df['Volume']).cumsum()
    df['Cumulative_Volume'] = df['Volume'].cumsum()
    df['VWAP'] = df['Cumulative_TP_Volume'] / df['Cumulative_Volume']
    return df

# ---- Signal Generators ---- #

def signal_bollinger(df):
    df['BB_SIGNAL'] = 0
    df.loc[df['Close'] < df['Lower_Band'], 'BB_SIGNAL'] = 1
    df.loc[df['Close'] > df['Upper_Band'], 'BB_SIGNAL'] = -1
    print('Bollinger Bands buy signals:', (df['BB_SIGNAL'] == 1).sum())
    print('Bollinger Bands sell signals:', (df['BB_SIGNAL'] == -1).sum())
    return df

def signal_macd(df):
    df['MACD_SIGNAL_FLAG'] = 0
    df.loc[df['MACD'] > df['signalline'], 'MACD_SIGNAL_FLAG'] = 1
    df.loc[df['MACD'] < df['signalline'], 'MACD_SIGNAL_FLAG'] = -1
    print('MACD buy signals:', (df['MACD_SIGNAL_FLAG'] == 1).sum())
    print('MACD sell signals:', (df['MACD_SIGNAL_FLAG'] == -1).sum())
    return df

def signal_rsi(df, overbought=70, oversold=30):
    df['RSI_SIGNAL'] = 0
    df.loc[df['RSI'] < oversold, 'RSI_SIGNAL'] = 1
    df.loc[df['RSI'] > overbought, 'RSI_SIGNAL'] = -1
    print('RSI buy signals:', (df['RSI_SIGNAL'] == 1).sum())
    print('RSI sell signals:', (df['RSI_SIGNAL'] == -1).sum())
    return df

def signal_stochastic(df, overbought=80, oversold=20):
    df['STOCH_SIGNAL'] = 0
    df.loc[(df['%K'] < oversold) & (df['%K'] > df['%D']), 'STOCH_SIGNAL'] = 1
    df.loc[(df['%K'] > overbought) & (df['%K'] < df['%D']), 'STOCH_SIGNAL'] = -1
    print('Stochastic buy signals:', (df['STOCH_SIGNAL'] == 1).sum())
    print('Stochastic sell signals:', (df['STOCH_SIGNAL'] == -1).sum())
    return df

def signal_atr(df, threshold=1.5):
    df['ATR_SIGNAL'] = 0
    df.loc[df['ATR'] > threshold, 'ATR_SIGNAL'] = 1
    print('ATR high volatility signals:', (df['ATR_SIGNAL'] == 1).sum())
    return df

def signal_vwap(df):
    df['VWAP_SIGNAL'] = 0
    df.loc[df['Close'] > df['VWAP'], 'VWAP_SIGNAL'] = 1
    df.loc[df['Close'] < df['VWAP'], 'VWAP_SIGNAL'] = -1
    print('VWAP buy signals:', (df['VWAP_SIGNAL'] == 1).sum())
    print('VWAP sell signals:', (df['VWAP_SIGNAL'] == -1).sum())
    return df


df = add_bollinger_bands(df)
df = add_macd(df)
df = add_rsi(df)
df = add_stochastic(df)
df = add_atr(df)
df = add_vwap(df)

df = signal_bollinger(df)
df = signal_macd(df)
df = signal_rsi(df)
df = signal_stochastic(df)
df = signal_vwap(df)


df = df.dropna().drop(columns=['Dividends', 'Stock Splits'])
df.head()





Bollinger Bands buy signals: 38
Bollinger Bands sell signals: 50
MACD buy signals: 433
MACD sell signals: 318
RSI buy signals: 101
RSI sell signals: 95
Stochastic buy signals: 55
Stochastic sell signals: 64
VWAP buy signals: 238
VWAP sell signals: 514


,Open,High,Low,Close,Volume,MA20,Upper_Band,Lower_Band,EMA12,EMA26,...,ATR,TP,Cumulative_TP_Volume,Cumulative_Volume,VWAP,BB_SIGNAL,MACD_SIGNAL_FLAG,RSI_SIGNAL,STOCH_SIGNAL,VWAP_SIGNAL
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-31 00:00:00-05:00,21.301102,21.799698,21.254936,21.762764,5686800,22.309373,24.640873,19.977874,21.638615,22.243138,...,0.724810,21.605799,5.292232e+09,238120700,22.224999,0,-1,0,0,-1
2022-02-01 00:00:00-05:00,21.707365,21.818163,21.458067,21.716599,10309700,22.229044,24.523889,19.934199,21.650613,22.204135,...,0.724150,21.664276,5.515585e+09,248430400,22.201729,0,-1,0,0,-1
2022-02-02 00:00:00-05:00,21.910498,22.021297,21.790466,21.947432,9033400,22.167181,24.419876,19.914487,21.696277,22.185120,...,0.655561,21.919732,5.713594e+09,257463800,22.191835,0,-1,0,0,-1
2022-02-03 00:00:00-05:00,21.310337,21.476536,20.987173,21.014874,12135000,22.096085,24.402075,19.790096,21.591446,22.098435,...,0.703706,21.159528,5.970365e+09,269598800,22.145370,0,-1,0,0,-1
2022-02-04 00:00:00-05:00,21.208773,21.430370,20.922542,21.134907,18253100,22.039763,24.383473,19.696053,21.521209,22.027063,...,0.693153,21.162607,6.356648e+09,287851900,22.083051,0,-1,1,0,-1
